In [8]:
import os
import json
import glob

def process_output_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        try:
            data = json.load(f)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in file {file_path}: {e}")
            return

    modified_data = []
    for entry in data:
        new_entry = {}
        flag = False
        # 处理 "filename"
        if "filename" in entry:
            raw_filename = os.path.basename(entry["filename"])
            new_entry["raw"] = raw_filename
        
        if "raw" in entry:
            raw_filename = os.path.basename(entry["raw"])
            new_entry["raw"] = raw_filename


        # 处理 "filenameNonSpe"
        if "filenameNonSpe" in entry:
            raw_non_spe_filename = os.path.basename(entry["filenameNonSpe"])
            new_entry["rawNonSpe"] = raw_non_spe_filename
        
        if "rawNonSpe" in entry:
            raw_non_spe_filename = os.path.basename(entry["rawNonSpe"])
            new_entry["rawNonSpe"] = raw_non_spe_filename

        
        if "filenameFirstPB" in entry:
            raw_first_pb_filename = os.path.basename(entry["filenameFirstPB"])
            new_entry["raw"] = raw_first_pb_filename
            flag = True


        if "filenameNonSpeFirstPB" in entry:
            raw_non_spe_first_pb_filename = os.path.basename(entry["filenameNonSpeFirstPB"])
            new_entry["rawNonSpe"] = raw_non_spe_first_pb_filename


        # 保留 "radius" 字段
        if "radius" in entry:
            if flag:
                new_entry["radius"] = 0
            else:
                new_entry["radius"] = entry["radius"]

        # 添加 "vmf" 字段
        # 假设 "vmf_parameters" 的编号与 "rawdataAll" 文件对应
        # 例如 rawdataAll0.bin -> vmf_parameters0.pth
        # 从 raw_filename 中提取编号
        base_name = os.path.splitext(raw_filename)[0]  # 去除扩展名
        # 提取数字编号，例如 "rawdataAll0" -> "0"
        import re
        match = re.search(r'\d+', base_name)
        if flag:
            vmf_filename = "vmf_parameters.pth"
        elif match:
            index = match.group()
            vmf_filename = f"vmf_parameters{index}.pth"
        else:
            vmf_filename = "vmf_parameters.pth"  # 默认值
        new_entry["vmf"] = vmf_filename

        modified_data.append(new_entry)

    # 将修改后的数据写回文件
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(modified_data, f, indent=4)
    print(f"Processed and updated file: {file_path}")





In [9]:
# 定义 foam14 目录的路径
foam14_dir = "..\\data\\foam14FirstShell"
# 查找所有子目录中的 output.json 文件
pattern = os.path.join(foam14_dir, "*", "output.json")
json_files = glob.glob(pattern, recursive=True)
for json_file in json_files:
    process_output_json(json_file)

Processed and updated file: ..\data\foam14FirstShell\0\output.json
Processed and updated file: ..\data\foam14FirstShell\1\output.json
Processed and updated file: ..\data\foam14FirstShell\10\output.json
Processed and updated file: ..\data\foam14FirstShell\11\output.json
Processed and updated file: ..\data\foam14FirstShell\12\output.json
Processed and updated file: ..\data\foam14FirstShell\13\output.json
Processed and updated file: ..\data\foam14FirstShell\14\output.json
Processed and updated file: ..\data\foam14FirstShell\15\output.json
Processed and updated file: ..\data\foam14FirstShell\16\output.json
Processed and updated file: ..\data\foam14FirstShell\17\output.json
Processed and updated file: ..\data\foam14FirstShell\18\output.json
Processed and updated file: ..\data\foam14FirstShell\19\output.json
Processed and updated file: ..\data\foam14FirstShell\2\output.json
Processed and updated file: ..\data\foam14FirstShell\20\output.json
Processed and updated file: ..\data\foam14FirstShel

In [3]:
import json
import os
import numpy as np
import math
import matplotlib.pyplot as plt

# 加载 ray 数据的函数
def load_raydata(filename, sizes, verbose=False, dtype=np.float32):
    # 加载数据
    rawdata = np.fromfile(filename, dtype=dtype)
    # print(filename, rawdata.shape)
    # 如果是 float16，转换为 float32
    if dtype == np.float16:
        rawdata = rawdata.astype(np.float32)
    rawdata = rawdata.reshape(-1, 4)
    num = rawdata.shape[0]
    x = rawdata[:, 0]
    phi = rawdata[:, 1] - np.pi
    r = np.sqrt(1 - x ** 2)
    y = r * np.cos(phi)
    z = r * np.sin(phi)

    # 创建网格分布
    x_edges = np.linspace(-1, 1, sizes[0] + 1)  
    phi_edges = np.linspace(-np.pi, np.pi, sizes[1] + 1)

    # 统计 (X, φ) 分布
    H, _, _ = np.histogram2d(x, phi, bins=[x_edges, phi_edges])
    ray_data = H.reshape(-1)
    if ray_data.shape[0] != sizes[0] * sizes[1]:
        print("Error: ray data shape mismatch!")
        exit(1)
    area = 4 * math.pi / (ray_data.shape[0])
    ray_data = ray_data

    
    return ray_data, num

# 从指定目录加载数据
def load_data_from_json(directory, json_filename, sizes, verbose=False, dtype=np.float32):
    json_path = os.path.join(directory, json_filename)
    
    # 读取 JSON 文件
    with open(json_path, 'r') as file:
        data_entries = json.load(file)
    
    # 逐条处理 JSON 数据
    all_data = []
    for entry in data_entries:
        raw_file_path = os.path.join(directory, entry['raw'])
        raw_nonspe_file_path = os.path.join(directory, entry['rawNonSpe'])
        
        if os.path.exists(raw_file_path) and os.path.exists(raw_nonspe_file_path):
            ray_data, all_num = load_raydata(raw_file_path, sizes, verbose, dtype)
            ray_nonspe_data, nonspe_num = load_raydata(raw_nonspe_file_path, sizes, verbose, dtype)
            all_data.append({
                'ray_data': ray_data,
                'ray_nonspe_data': ray_nonspe_data,
                'radius': entry['radius'],
                'ratio': nonspe_num/all_num
            })
        else:
            print(f"Warning: {raw_file_path} does not exist.")
    
    return all_data

def plot_outputs_3d(references, predictions, sizes, save_path=None,title=""):
    # 定义 z 和 phi 的范围
    z_min, z_max = -1, 1
    phi_min, phi_max = -np.pi, np.pi

    # 创建用于 3D 绘图的网格
    z_in = np.linspace(z_min, z_max, sizes[0])
    phi_in = np.linspace(phi_min, phi_max, sizes[1])

    Z, Phi = np.meshgrid(z_in, phi_in, indexing='ij')

    target_img = references
    predict_img = predictions

    # 确保输入数据的形状与网格匹配
    if predict_img.shape != Z.shape:
        predict_img = predict_img.reshape(Z.shape)
    if target_img is not None and target_img.shape != Z.shape:
        target_img = target_img.reshape(Z.shape)

    # 设置用于 3D 可视化的子图
    if target_img is not None and np.sum(target_img) > 0:
        fig = plt.figure(figsize=(14, 6))
        ax1 = fig.add_subplot(1, 2, 1, projection='3d')
        ax2 = fig.add_subplot(1, 2, 2, projection='3d')
    else:
        fig = plt.figure(figsize=(10, 6))
        ax1 = fig.add_subplot(1, 1, 1, projection='3d')

    # 绘制预测数据的 3D 曲面
    ax1.plot_surface(Z, Phi, predict_img, rstride=1, cstride=1, cmap='rainbow')
    ax1.set_title(f'W/O Specular {title}')
    ax1.set_xlabel('Z')
    ax1.set_ylabel('Phi')
    ax1.set_zlabel('Value')

    # 如果有参考数据，绘制其 3D 曲面
    if target_img is not None and np.sum(target_img) > 0:
        ax2.plot_surface(Z, Phi, target_img, rstride=1, cstride=1, cmap='rainbow')
        ax2.set_title(f'ALL {title}')
        ax2.set_xlabel('Z')
        ax2.set_ylabel('Phi')
        ax2.set_zlabel('Value')

    # 显示图形
    plt.tight_layout()

    if save_path is not None:
        plt.savefig(save_path)
        plt.close()
    else:
        plt.show()

def plot_outputs_heatmap(references, predictions, sizes, save_path=None, title=""):
    # 定义 z 和 phi 的范围
    z_min, z_max = -1, 1
    phi_min, phi_max = -np.pi, np.pi

    # 创建用于绘制的网格
    z_in = np.linspace(z_min, z_max, sizes[0])
    phi_in = np.linspace(phi_min, phi_max, sizes[1])

    target_img = references
    predict_img = predictions

    # 确保输入数据的形状与网格匹配
    if predict_img.shape != (sizes[0], sizes[1]):
        predict_img = predict_img.reshape((sizes[0], sizes[1]))
    if target_img is not None and target_img.shape != (sizes[0], sizes[1]):
        target_img = target_img.reshape((sizes[0], sizes[1]))

    # 设置用于热力图的子图
    if target_img is not None and np.sum(target_img) > 0:
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        ax1, ax2 = axes
    else:
        fig, ax1 = plt.subplots(1, 1, figsize=(10, 6))

    # 绘制预测数据的热力图
    im1 = ax1.imshow(predict_img, extent=[phi_min, phi_max, z_min, z_max], origin='lower', aspect='auto', cmap='rainbow')
    ax1.set_title(f'W/O Specular {title}')
    ax1.set_xlabel('Phi')
    ax1.set_ylabel('Z')
    fig.colorbar(im1, ax=ax1, orientation='vertical')

    # 如果有参考数据，绘制其热力图
    if target_img is not None and np.sum(target_img) > 0:
        im2 = ax2.imshow(target_img, extent=[phi_min, phi_max, z_min, z_max], origin='lower', aspect='auto', cmap='rainbow')
        ax2.set_title(f'ALL {title}')
        ax2.set_xlabel('Phi')
        ax2.set_ylabel('Z')
        fig.colorbar(im2, ax=ax2, orientation='vertical')

    # 显示图形
    plt.tight_layout()

    if save_path is not None:
        plt.savefig(save_path)
        plt.close()
    else:
        plt.show()


In [6]:
directory = "D:/Downloads/foam14FirstShell/1"
all_data = load_data_from_json(directory, 'output.json', [64, 64], verbose=True, dtype=np.float16)

In [7]:
os.makedirs(os.path.join(directory, "img"), exist_ok=True)
for data in all_data:
    ray_data = data['ray_data']
    ray_nonspe_data = data['ray_nonspe_data']
    radius = data['radius']
    save_path = os.path.join(directory, f"img/radius_{radius:.1f}.png")
    # 绘制 3D 图形
    plot_outputs_3d(ray_data, ray_nonspe_data, [64, 64], title=f"Radius {radius:.1f}", save_path=save_path)
    print(f"radius: {radius:.1f}, nonspe/all: {data['ratio']:.2e}")
    

radius: 0.9, nonspe/all: 9.45e-01
radius: 0.8, nonspe/all: 9.45e-01
radius: 0.7, nonspe/all: 9.45e-01
radius: 0.6, nonspe/all: 9.45e-01
radius: 0.5, nonspe/all: 9.45e-01
radius: 0.4, nonspe/all: 9.45e-01
radius: 0.2, nonspe/all: 9.45e-01


In [ ]:
os.makedirs(os.path.join(directory, "img"), exist_ok=True)
for data in all_data:
    ray_data = data['ray_data']
    ray_nonspe_data = data['ray_nonspe_data']
    radius = data['radius']
    save_path = os.path.join(directory, f"img/heatmap_radius_{radius:.1f}.png")
    # 绘制 2D 图形
    plot_outputs_heatmap(ray_data, ray_nonspe_data, [64, 64], title=f"Radius {radius:.1f}", save_path=save_path)